In [1]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps, operator_average
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t_spin, get_h1_spin, get_g_spin, get_t
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization, remain_only_tridiagonal_elements
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt,  generate_krylov_space_othogonal_against
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [2]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/NH3/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/NH3/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 36)
(36, 36)
rl errV: 6.421105521729929e-13
abs errV: 4.286591936668418e-12
errt: 1.0574710399920436e-13
errh: 1.8088404989173088e-14
errht: 9.006422291220954e-14


In [3]:
# H_correct = get_molecular_Hamiltonian_as_sparse_matrix(get_h1_spin(h1), get_g_spin(eri))
# H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H_correct_NH3.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

In [4]:
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

These Hamiltonian are exact molecular Hamiltonian and molecular Hamiltonian reconstructed by THC tensors. The calculation cost time, so that we store them in disk and load them when needed. For water molecule H2O in STO-6G basis, the error is small for r_THC = 28.

Actually, considering there are always 10 electrons for a water molecule, we only retain the elements which operator quantum states with 10 electrons.

In [5]:
e_ground = -67.46923609481743
e_1st_ex = -67.00839085980371
e_2nd_ex = -67.00038986115365

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [6]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 62, 100, 154, 100, 62, 16, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [7]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state.

In [8]:
filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_1st_excited.pkl"
with open(filename, 'rb') as file:
    initial = pickle.load(file)

print(operator_average(initial, mpo_ref) - e_1st_ex)

(0.00929712659700499+0j)


In [9]:
filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_ground_state.pkl"
with open(filename, 'rb') as file:
    NH3_ground = pickle.load(file)
    
print(operator_average(NH3_ground, mpo_ref) - e_ground)    

(2.747419941329099e-06+0j)


MAYBE: for each new-generated Krylov, we delete the component of ground state and then store it.
When approximate state is not available, orthogonalize a initial state to all previous ground/excited states.
实在不行：用HFS强行和基态正交再找激发态。

generate Krylov space:

In [11]:
N_Krylov_1 = 45
psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 120
trunc_tol = 0
foldername_1 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex" 
#generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)
generate_krylov_space_othogonal_against(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1, copy.deepcopy(NH3_ground))
#generate_krylov_space_othogonal_against(N_Krylov, H_mu_nu_list_spin_layer, psi_original, max_bond_Krylov, trunc_tol, r_THC, foldername, vec_to_remove)

In [12]:
#或许在这一步生成GS的时候也把ground state考虑进来
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [13]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_1st_ex, mpo_ref)

(0.008924045479787424+0j)
(0.006944921294376627+0j)
(0.004542685518458711+0j)
(0.0035258704237861593+0j)
(0.0028410827820266604+0j)
(0.0023823890522578495+0j)
(0.0021159262222312236+0j)
(0.00182515476662104+0j)
(0.0015054096524380611+0j)


In [ ]:
# (0.008924045400320324+0j)
# (0.006944921621780509+0j)
# (0.004542685286736514+0j)
# (0.003525847583532027+0j)
# (0.0028409863732150598+0j)
# (0.002382367171549049+0j)
# (0.0021164262737585204+0j)
# (0.0018315294370978563+0j)
# (0.00152519022194042+0j)
# (0.001436938691853129+0j)

In [ ]:
# spin = 0
# for i in range (L):
#     if i%2 == 0:
#         spin_local = 0.5* operator_average(v_ritz_1, get_num_op_mpo(i, L))
#     if i%2 == 1:
#         spin_local = -0.5* operator_average(v_ritz_1, get_num_op_mpo(i, L))
#     spin += spin_local 
# print(spin)

Restart:

In [ ]:
N_Krylov_2 = 50
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 120
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_1"
generate_krylov_space_othogonal_against(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2, copy.deepcopy(NH3_ground))

[1, 4, 16, 64, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-6.77047505470435e-06+0j)
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.2933849698838579e-06+0j)
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.772016248569063e-07+0j)
5
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(2.0990392024621059e-07+0j)
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-3.885470724582259e-06+0j)
7
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(2.107521030046309e-08+0j)
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-6.509714558195473e-07+0j)
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.448180842619225e-07+0j)
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(9.733482179195114e-08+0j)
11
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(1.0429497301813612e-08+0j)
12
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-4.6068225919828005e-07+0j)
13
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(4.149838447702059e-09+0j)
14
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.6312851680045108e-07+0j)
15
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.7616285503729628e-07+0j)
16
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(4.532341180791752e-08+0j)
17

In [ ]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
#H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

In [ ]:
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)

In [ ]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_1st_ex, mpo_ref)

(0.001397955184486932+0j)
(0.0013183691311979828+0j)
(0.0012464339127689072+0j)
(0.0011965522737682477+0j)
(0.0011858446152643864+0j)
(0.0011833601004553884+0j)
(0.0011826279005475726+0j)
(0.0011798755404157646+0j)
(0.0011749765497484077+0j)
(0.001154484244935361+0j)


Restart:

In [ ]:
N_Krylov_3 = 30
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 120
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_2"
generate_krylov_space_othogonal_against(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3, copy.deepcopy(NH3_ground))


[1, 4, 16, 64, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(8.353112347673614e-06+0j)
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(6.9899508935772e-06+0j)
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-4.816275200445849e-07+0j)
5
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-8.118258827661227e-08+0j)
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.1949016067989065e-06+0j)
7
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-4.1905757771203167e-08+0j)
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.673782543077859e-08+0j)
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-7.235602668192255e-07+0j)
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.340514967856774e-08+0j)
11
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(4.103633266679639e-08+0j)
12
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(8.273274449644102e-07+0j)
13
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.6561415740320648e-09+0j)
14
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(4.113574233793253e-07+0j)
15
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-8.206539858778038e-07+0j)
16
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(1.0553350940042169e-08+0j)
17
[

In [ ]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)


In [ ]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_1st_ex, mpo_ref)

(0.0011470223215042097+0j)
(0.0011260387373823733+0j)
(0.0011241968222037713+0j)
(0.0011203298214326196+0j)


(0.0011112679826084104+0j)
(0.001103322100689752+0j)


restart:

In [ ]:
N_Krylov_4 = 30
psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 120
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_3"
generate_krylov_space_othogonal_against(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4, copy.deepcopy(NH3_ground))


[1, 4, 16, 64, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-9.04424251187529e-07+0j)
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-9.089688248584515e-07+0j)
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(4.5949291244500837e-07+0j)
5
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-3.944863132818875e-07+0j)
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(5.490490026248179e-07+0j)
7
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-7.567945349207434e-08+0j)
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(1.8574054452423407e-08+0j)
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-9.074157716841406e-08+0j)
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-3.435466685707423e-10+0j)
11
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.0212469814187448e-07+0j)
12
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.3604932507102014e-06+0j)
13
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-4.356213597839025e-09+0j)
14
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.4772839981265572e-06+0j)
15
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(2.870208327517961e-07+0j)
16
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(1.1956536341109253e-07+0j)


In [ ]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)

In [ ]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_1st_ex, mpo_ref)

(0.0010998118856093697+0j)
(0.0010968866635892027+0j)


(0.0010942610815902754+0j)
(0.0010934011441037228+0j)
(0.0010919006127920738+0j)
(0.0010755484506006496+0j)


restart:

In [ ]:
N_Krylov_5 = 30
psi_original_5 = copy.deepcopy(v_ritz_4)
max_bond_Krylov_5 = 120
trunc_tol = 0
foldername_5 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_4"
generate_krylov_space_othogonal_against(N_Krylov_5, H_mu_nu_list_spin_layer, psi_original_5, max_bond_Krylov_5, trunc_tol, r_thc, foldername_5, copy.deepcopy(NH3_ground))

[1, 4, 16, 64, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.1714048211014957e-05+0j)
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-3.468066074480815e-06+0j)
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(6.886328102325909e-07+0j)
5
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.4154127993493354e-08+0j)
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(9.736887697931861e-08+0j)
7
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(6.65531486721666e-09+0j)
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-2.9969235399237995e-08+0j)
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.087406012742564e-06+0j)
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-2.8024365038685777e-07+0j)
11
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.5136955011210694e-08+0j)
12
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(2.246726629742614e-07+0j)
13
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(1.3225036972998512e-08+0j)
14
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(7.971523854286835e-09+0j)
15
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(2.304633255564045e-07+0j)
16
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.899711602542601e-07+0j)
17


In [ ]:
H_reduced_non_rotho_5 = generate_reduced_H_non_ortho(N_Krylov_5, foldername_5, mpo_ref)
coeff_5 = coeff_gram_schmidt(N_Krylov_5, foldername_5)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_5 = np.einsum('ik, kl, jl -> ij', coeff_5.conj(), H_reduced_non_rotho_5, coeff_5)

In [ ]:
e_ritz_5, v_ritz_5 = solve_ritz(foldername_5, H_reduced_5, N_Krylov_5, coeff_5, max_bond_Krylov_5, e_1st_ex, mpo_ref)

(0.0010687932477679851+0j)
(0.0010643892557027357+0j)
(0.001063453286022309+0j)
(0.0010632522551645707+0j)
(0.0010630885972489068+0j)


(0.0010598104073693548+0j)


In [ ]:
N_Krylov_6 = 30
psi_original_6 = copy.deepcopy(v_ritz_5)
max_bond_Krylov_6 = 120
trunc_tol = 0
foldername_6 = f"/work_fast/ge49cag/code_datas/NH3_1st_ex_restart_5"
generate_krylov_space_othogonal_against(N_Krylov_6, H_mu_nu_list_spin_layer, psi_original_6, max_bond_Krylov_6, trunc_tol, r_thc, foldername_6, copy.deepcopy(NH3_ground))

[1, 4, 16, 64, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-7.765123833580649e-06+0j)
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.8931886490041909e-06+0j)
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-5.502639112260296e-07+0j)
5
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-4.3133040821631335e-09+0j)
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.8187130070257103e-07+0j)
7
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.2350674923418045e-08+0j)
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(5.611878675267892e-09+0j)
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-4.370169294692089e-09+0j)
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-1.5058555171740777e-08+0j)
11
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(1.966361544850293e-08+0j)
12
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-3.7441013605670175e-07+0j)
13
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(2.0963182562993947e-08+0j)
14
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-2.6248132087796196e-07+0j)
15
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(3.361750166233098e-07+0j)
16
[1, 4, 16, 64, 120, 64, 16, 4, 1]
(-9.027061427419487e-08

KeyboardInterrupt: 

In [ ]:
H_reduced_non_rotho_6 = generate_reduced_H_non_ortho(N_Krylov_6, foldername_6, mpo_ref)
coeff_6 = coeff_gram_schmidt(N_Krylov_6, foldername_6)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_6 = np.einsum('ik, kl, jl -> ij', coeff_6.conj(), H_reduced_non_rotho_6, coeff_6)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:214: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:113: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [ ]:
e_ritz_6, v_ritz_6 = solve_ritz(foldername_6, H_reduced_6, N_Krylov_6, coeff_6, max_bond_Krylov_6, e_1st_ex, mpo_ref)

(6.142353470295348e-07+0j)
(5.389562574009688e-07+0j)
(5.135676417467039e-07+0j)
(4.959903492363082e-07+0j)
(4.883235931174568e-07+0j)
(4.873014063377923e-07+0j)
(4.842666498916515e-07+0j)
(4.822072270371791e-07+0j)
(4.819374055387016e-07+0j)
(4.808381532939165e-07+0j)
